# Data Cleaning, Data Merging

## Merged data output available with last date, on Github


The process will be made as .py file afterwards

===================================================

---

# Data Cleaning 1: Population dataset

ACS, 2018

In [1]:
import pandas as pd
import numpy as np
import re
import os
import datetime as dt

In [48]:
# read in population to merge it with output data
pop = pd.read_csv('https://raw.githubusercontent.com/athvedt/KPMG-Team/main/Data/acs_pop_state.csv?token=ANFRA3A6SG3IYQTZERZA7SS7W3FTW',
                  dtype = str).drop(['variable'], axis=1)
pop['estimate'] = pop['estimate'].astype(int)

pop = pop.rename(columns={"GEOID": "State_FIPS", "NAME": "StateName", "estimate": "population"})
pop.head()

,State_FIPS,StateName,population
0,01,Alabama,4864680
1,02,Alaska,738516
2,04,Arizona,6946685
3,05,Arkansas,2990671
4,06,California,39148760


---

# Data Cleaning 2: Oxford Policy Dataset (US ONLY)

https://raw.githubusercontent.com/OxCGRT/USA-covid-policy/master/data/OxCGRT_US_latest.csv

Dataset information:
* Aggregated by state
* Reported daily

Columns:
57 columns in total

23
C1_School closing,C1_Flag,C1_Notes,
C2_Workplace closing,C2_Flag,C2_Notes,
C3_Cancel public events,C3_Flag,C3_Notes,
C4_Restrictions on gatherings,C4_Flag,C4_Notes,
C5_Close public transport,C5_Flag,C5_Notes,
C6_Stay at home requirements,C6_Flag,C6_Notes,
C7_Restrictions on internal movement,C7_Flag,C7_Notes,
C8_International travel controls,C8_Notes,


9
E1_Income support,E1_Flag,E1_Notes,
E2_Debt/contract relief,E2_Notes,
E3_Fiscal measures,E3_Notes,
E4_International support,E4_Notes,


11
H1_Public information campaigns,H1_Flag,H1_Notes,
H2_Testing policy,H2_Notes,
H3_Contact tracing,H3_Notes,
H4_Emergency investment in healthcare,H4_Notes,
H5_Investment in vaccines,H5_Notes,

2
M1_Wildcard,M1_Notes,

12
ConfirmedCases,ConfirmedDeaths,
StringencyIndex,StringencyIndexForDisplay,
StringencyLegacyIndex,StringencyLegacyIndexForDisplay,
GovernmentResponseIndex,GovernmentResponseIndexForDisplay,
ContainmentHealthIndex,ContainmentHealthIndexForDisplay,
EconomicSupportIndex,EconomicSupportIndexForDisplay

In [22]:
# Read in the data
oxford_raw = pd.read_csv('https://raw.githubusercontent.com/OxCGRT/USA-covid-policy/master/data/OxCGRT_US_latest.csv')

In [23]:
oxford_raw.head()

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C1_Notes,C2_Workplace closing,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,United States,USA,NaN,NaN,NAT_GOV,20200101,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,United States,USA,NaN,NaN,NAT_GOV,20200101,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,United States,USA,NaN,NaN,NAT_GOV,20200102,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,United States,USA,NaN,NaN,NAT_GOV,20200102,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,United States,USA,NaN,NaN,NAT_GOV,20200103,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
# Drop columns not necessary
# only need two columns

oxford = oxford_raw[['RegionName', 'RegionCode', 'Jurisdiction', 'Date', 'StringencyIndexForDisplay', 'ContainmentHealthIndexForDisplay']]
oxford.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17118 entries, 0 to 17117
Data columns (total 6 columns):
RegionName                          16484 non-null object
RegionCode                          16484 non-null object
Jurisdiction                        17118 non-null object
Date                                17118 non-null int64
StringencyIndexForDisplay           16761 non-null float64
ContainmentHealthIndexForDisplay    16761 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 802.5+ KB


In [25]:
# data transformation
# state only
# non-null states
oxford = oxford[(oxford['Jurisdiction']!='NAT_GOV') & (oxford['RegionName'].notnull())]
oxford['Date'] = oxford['Date'].apply(lambda x: dt.datetime.strptime(str(x), '%Y%m%d'))
oxford['RegionCode'] = [re.sub('US_','', i) for i in oxford['RegionCode']]
oxford = oxford.drop('Jurisdiction', axis=1)
oxford = oxford.rename(columns={'StringencyIndexForDisplay': 'stringency', 'ContainmentHealthIndexForDisplay': 'containment'})

In [26]:
oxford.head()

,RegionName,RegionCode,Date,stringency,containment
634,Alaska,AK,2020-01-01,0.0,0.0
635,Alaska,AK,2020-01-02,0.0,0.0
636,Alaska,AK,2020-01-03,0.0,0.0
637,Alaska,AK,2020-01-04,0.0,0.0
638,Alaska,AK,2020-01-05,0.0,0.0


In [27]:
# sanity check: end of March in NY - everything should be closed
oxford[(oxford['Date']=='2020-03-31')&(oxford['RegionName']=='New York')]

,RegionName,RegionCode,Date,stringency,containment
11502,New York,NY,2020-03-31,79.63,66.67


In [28]:
# sanity check: Most recent in NY - everything should be closed
oxford[(oxford['Date']==max(oxford['Date']))&(oxford['RegionName']=='New York')]

,RegionName,RegionCode,Date,stringency,containment
11728,New York,NY,2020-11-12,69.91,75.35


---

# Data Cleaning 3: JHU COVID-19 US Dataset

Data cleaning of JHU COVID-19 USA dataset

Aggregated by state

* Confirmed = cumulative confirmed
* Deaths = cumulative deaths
* Date
* new_confirmed = new confirmed cases each day
* new_deaths = new death cases each day
* new_confirmed_7 = rollling average, 7 days
* new_deaths_7 = rolling average, 7 days
* new_confirmed_14 = rolling average, 14 days
* new_deaths_14 = rolling average, 14 days

In [29]:
raw_cases_us = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
raw_deaths_us = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")

In [30]:
raw_cases_us.tail()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20
3335,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,...,847,862,873,897,922,932,977,1007,1022,1025
3336,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,...,492,510,518,529,542,544,551,568,614,627
3337,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,...,0,2,0,0,0,0,0,0,0,0
3338,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,...,160,167,175,179,188,185,187,190,197,199
3339,84056045,US,USA,840,56045.0,Weston,Wyoming,US,43.839612,-104.567488,...,169,181,189,203,294,296,306,310,321,323


In [31]:
raw_deaths_us.tail()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20
3335,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,...,1,2,2,2,2,2,2,2,2,2
3336,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,...,3,3,3,3,3,3,4,4,4,4
3337,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,...,10,0,0,0,0,9,0,0,0,0
3338,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,...,7,7,7,7,7,7,7,7,7,7
3339,84056045,US,USA,840,56045.0,Weston,Wyoming,US,43.839612,-104.567488,...,0,0,1,1,1,1,0,0,0,0


In [44]:
# clean us cases and deaths
def us_col_clean(case_df, death_df):
    cols_to_drop = ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Country_Region', 'Lat', 'Long_', 'Combined_Key']
    tmp_case = case_df.drop(cols_to_drop, axis=1)
    tmp_death = death_df.drop(cols_to_drop + ['Population'], axis=1)
    tmp_case['indicator'] = 'Confirmed'
    tmp_death['indicator'] = 'Deaths'
    tmp = pd.concat([tmp_case, tmp_death], axis=0, ignore_index=True)
    return tmp

In [45]:
us_raw = us_col_clean(raw_cases_us, raw_deaths_us)

us_raw.tail()

,Province_State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,indicator
6675,Wyoming,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,Deaths
6676,Wyoming,0,0,0,0,0,0,0,0,0,...,3,3,3,3,3,4,4,4,4,Deaths
6677,Wyoming,0,0,0,0,0,0,0,0,0,...,0,0,0,0,9,0,0,0,0,Deaths
6678,Wyoming,0,0,0,0,0,0,0,0,0,...,7,7,7,7,7,7,7,7,7,Deaths
6679,Wyoming,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,0,0,0,0,Deaths


In [34]:
def us_shape_clean(df):
    df = pd.melt(df, id_vars=['Province_State', 'indicator'], var_name='Date', value_name='Value')
    #df.head()
    df['Date'] = df['Date'].apply(lambda x: dt.datetime.strptime(str(x), '%m/%d/%y'))
    df = pd.pivot_table(df, index = ['Province_State', 'Date'], columns='indicator', values = 'Value', aggfunc=np.sum).reset_index()
    # Remove non-states
    not_state = ['American Samoa', 'Diamond Princess', 'Grand Princess', 'Guam', 'Northern Mariana Islands', 'Puerto Rico', 'Virgin Islands']
    df = df[~df['Province_State'].isin(not_state)]
    df = df.sort_values(by=['Date', 'Province_State'])

    return df

In [35]:
us_clean = us_shape_clean(us_raw)

In [36]:
us_clean.tail()

indicator,Province_State,Date,Confirmed,Deaths
15929,Virginia,2020-11-11,196506,3741
16224,Washington,2020-11-11,120011,2482
16519,West Virginia,2020-11-11,30201,553
16814,Wisconsin,2020-11-11,301349,2554
17109,Wyoming,2020-11-11,19374,127


In [37]:
def calculate_daily(df, state):
    df = df.sort_values(by=['Date'])
    confirmed = df[df['Province_State'] == state]['Confirmed'].values.tolist()
    deaths = df[df['Province_State'] == state]['Deaths'].values.tolist()
    tmp_df = df[df['Province_State'] == state]
    new_confirmed = [confirmed[0]]
    new_deaths = [deaths[0]]
    for i in range(len(tmp_df)-1):
        new_confirmed.append(confirmed[i+1]-confirmed[i])
        new_deaths.append(deaths[i+1]-deaths[i])
    tmp_df['new_confirmed'] = new_confirmed
    tmp_df['new_deaths'] = new_deaths

    return tmp_df # returns subsetted df with daily new confirmed and daily new deaths

In [38]:
states = sorted(set(us_clean['Province_State'].values.tolist()))
us_final = pd.DataFrame()

for state in states:
    tmp = calculate_daily(us_clean, state=state)
    us_final = us_final.append(tmp, ignore_index=True)

C:\Users\bolim\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\bolim\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [39]:
us_final.tail()

indicator,Province_State,Date,Confirmed,Deaths,new_confirmed,new_deaths
15040,Wyoming,2020-11-07,16597,114,192,9
15041,Wyoming,2020-11-08,17310,114,713,0
15042,Wyoming,2020-11-09,18010,114,700,0
15043,Wyoming,2020-11-10,19242,127,1232,13
15044,Wyoming,2020-11-11,19374,127,132,0


In [40]:
# Compute rolling 7 day average for new_confirmed and new_death for each state
us_final['new_confirmed_7'] = us_final.groupby('Province_State')['new_confirmed'].rolling(7).mean().reset_index(0, drop=True)
us_final['new_deaths_7'] = us_final.groupby('Province_State')['new_deaths'].rolling(7).mean().reset_index(0, drop=True)

# Comptue rolling 14 day avg for new_confirmed and new_deaths for each state
us_final['new_confirmed_14'] = us_final.groupby('Province_State')['new_confirmed'].rolling(14).mean().reset_index(0, drop=True)
us_final['new_deaths_14'] = us_final.groupby('Province_State')['new_deaths'].rolling(14).mean().reset_index(0, drop=True)

In [41]:
us_final.tail(10)

indicator,Province_State,Date,Confirmed,Deaths,new_confirmed,new_deaths,new_confirmed_7,new_deaths_7,new_confirmed_14,new_deaths_14
15035,Wyoming,2020-11-02,14167,97,444,10,384.285714,2.857143,346.857143,2.857143
15036,Wyoming,2020-11-03,14621,93,454,-4,402.142857,2.285714,363.928571,2.285714
15037,Wyoming,2020-11-04,15044,105,423,12,414.000000,4.000000,371.142857,3.142857
15038,Wyoming,2020-11-05,15409,105,365,0,414.571429,2.571429,377.857143,2.642857
15039,Wyoming,2020-11-06,16405,105,996,0,482.428571,2.571429,418.571429,2.642857
15040,Wyoming,2020-11-07,16597,114,192,9,471.285714,3.857143,413.714286,3.285714
15041,Wyoming,2020-11-08,17310,114,713,0,512.428571,3.857143,447.785714,3.285714
15042,Wyoming,2020-11-09,18010,114,700,0,549.000000,2.428571,466.642857,2.642857
15043,Wyoming,2020-11-10,19242,127,1232,13,660.142857,4.857143,531.142857,3.571429
15044,Wyoming,2020-11-11,19374,127,132,0,618.571429,3.142857,516.285714,3.571429


# Step 4: Merging

In [52]:
# ACS population with oxford data

merge1 = pd.merge(oxford, pop, left_on = 'RegionName', right_on='StateName', how='left')
merge1 = merge1.drop(['RegionName'], axis=1)
merge1.tail()

,RegionCode,Date,stringency,containment,State_FIPS,StateName,population
16479,WY,2020-11-08,42.59,52.78,56,Wyoming,581836.0
16480,WY,2020-11-09,42.59,52.78,56,Wyoming,581836.0
16481,WY,2020-11-10,42.59,52.78,56,Wyoming,581836.0
16482,WY,2020-11-11,42.59,52.78,56,Wyoming,581836.0
16483,WY,2020-11-12,42.59,52.78,56,Wyoming,581836.0


In [53]:
len(oxford) == len(merge1) # sanity check

True

In [54]:
# Merge oxford - StateName, Date & us_final - Province_State, Date

merge2 = pd.merge(us_final, merge1, left_on=['Province_State', 'Date'], right_on=['StateName', 'Date'], how='left')
merge2.tail()

,Province_State,Date,Confirmed,Deaths,new_confirmed,new_deaths,new_confirmed_7,new_deaths_7,new_confirmed_14,new_deaths_14,RegionCode,stringency,containment,State_FIPS,StateName,population
15040,Wyoming,2020-11-07,16597,114,192,9,471.285714,3.857143,413.714286,3.285714,WY,42.59,52.78,56,Wyoming,581836.0
15041,Wyoming,2020-11-08,17310,114,713,0,512.428571,3.857143,447.785714,3.285714,WY,42.59,52.78,56,Wyoming,581836.0
15042,Wyoming,2020-11-09,18010,114,700,0,549.000000,2.428571,466.642857,2.642857,WY,42.59,52.78,56,Wyoming,581836.0
15043,Wyoming,2020-11-10,19242,127,1232,13,660.142857,4.857143,531.142857,3.571429,WY,42.59,52.78,56,Wyoming,581836.0
15044,Wyoming,2020-11-11,19374,127,132,0,618.571429,3.142857,516.285714,3.571429,WY,42.59,52.78,56,Wyoming,581836.0


In [56]:
len(merge2) == len(us_final)

True

In [57]:
merge2.head()

,Province_State,Date,Confirmed,Deaths,new_confirmed,new_deaths,new_confirmed_7,new_deaths_7,new_confirmed_14,new_deaths_14,RegionCode,stringency,containment,State_FIPS,StateName,population
0,Alabama,2020-01-22,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
1,Alabama,2020-01-23,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
2,Alabama,2020-01-24,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
3,Alabama,2020-01-25,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
4,Alabama,2020-01-26,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0


In [59]:
merge2.head(20)

,Province_State,Date,Confirmed,Deaths,new_confirmed,new_deaths,new_confirmed_7,new_deaths_7,new_confirmed_14,new_deaths_14,RegionCode,stringency,containment,State_FIPS,StateName,population
0,Alabama,2020-01-22,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
1,Alabama,2020-01-23,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
2,Alabama,2020-01-24,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
3,Alabama,2020-01-25,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
4,Alabama,2020-01-26,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
5,Alabama,2020-01-27,0,0,0,0,NaN,NaN,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
6,Alabama,2020-01-28,0,0,0,0,0.0,0.0,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
7,Alabama,2020-01-29,0,0,0,0,0.0,0.0,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
8,Alabama,2020-01-30,0,0,0,0,0.0,0.0,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0
9,Alabama,2020-01-31,0,0,0,0,0.0,0.0,NaN,NaN,AL,0.0,0.0,01,Alabama,4864680.0


In [60]:
## rolling 7 day cases and deaths, per 100K (100,000)
final = merge2.copy()
final['per_100k_new_confirmed_7'] = (final['new_confirmed_7']/final['population']) * 100000
final['per_100k_new_deaths_7'] = (final['new_deaths_7']/final['population']) * 100000

In [72]:
final.tail(10)

,Province_State,Date,Confirmed,Deaths,new_confirmed,new_deaths,new_confirmed_7,new_deaths_7,new_confirmed_14,new_deaths_14,RegionCode,stringency,containment,State_FIPS,StateName,population,per_100k_new_confirmed_7,per_100k_new_deaths_7
15035,Wyoming,2020-11-02,14167,97,444,10,384.285714,2.857143,346.857143,2.857143,WY,42.59,52.78,56,Wyoming,581836.0,66.047084,0.491056
15036,Wyoming,2020-11-03,14621,93,454,-4,402.142857,2.285714,363.928571,2.285714,WY,42.59,52.78,56,Wyoming,581836.0,69.116187,0.392845
15037,Wyoming,2020-11-04,15044,105,423,12,414.000000,4.000000,371.142857,3.142857,WY,42.59,52.78,56,Wyoming,581836.0,71.154071,0.687479
15038,Wyoming,2020-11-05,15409,105,365,0,414.571429,2.571429,377.857143,2.642857,WY,42.59,52.78,56,Wyoming,581836.0,71.252282,0.441951
15039,Wyoming,2020-11-06,16405,105,996,0,482.428571,2.571429,418.571429,2.642857,WY,42.59,52.78,56,Wyoming,581836.0,82.914871,0.441951
15040,Wyoming,2020-11-07,16597,114,192,9,471.285714,3.857143,413.714286,3.285714,WY,42.59,52.78,56,Wyoming,581836.0,80.999752,0.662926
15041,Wyoming,2020-11-08,17310,114,713,0,512.428571,3.857143,447.785714,3.285714,WY,42.59,52.78,56,Wyoming,581836.0,88.070964,0.662926
15042,Wyoming,2020-11-09,18010,114,700,0,549.000000,2.428571,466.642857,2.642857,WY,42.59,52.78,56,Wyoming,581836.0,94.356485,0.417398
15043,Wyoming,2020-11-10,19242,127,1232,13,660.142857,4.857143,531.142857,3.571429,WY,42.59,52.78,56,Wyoming,581836.0,113.458579,0.834796
15044,Wyoming,2020-11-11,19374,127,132,0,618.571429,3.142857,516.285714,3.571429,WY,42.59,52.78,56,Wyoming,581836.0,106.313708,0.540162


In [68]:
file_name = 'us_state_level_clean_' + str(max(final['Date'].dt.date)) +'.csv'

In [69]:
file_name

'us_state_level_clean_2020-11-11.csv'

In [71]:
final.to_csv(file_name) # export